# Resampling Methods

In [ ]:
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

## Train, Validation and Test Sets

Machine learning algorithms overfit taining data. Predictive performances **MUST** be evaluated on independant hold-out dataset.


![Train, Validation and Test Sets.](images/train_validation_test_sets.png)

1. **Training dataset**: Dataset used to fit the model (set the model parameters like weights). The *training error* can be easily calculated by applying the statistical learning method to the observations used in its training. But because of overfitting, the **training error rate can dramatically underestimate the error** that would be obtained on new samples.

2. **Validation dataset**: Dataset used to provide an unbiased evaluation of a model fit on the training dataset while **tuning model hyperparameters**. The validation error is the average error that results from a learning method to predict the response on a new (validation) samples that is, on samples that were not used in training the method. 

2. **Test Dataset**: Dataset used to provide an unbiased evaluation of a final model fit on the training dataset. It is only used once a model is completely trained(using the train and validation sets).

What is the Difference Between Test and Validation Datasets? by [Jason Brownlee](https://machinelearningmastery.com/difference-test-validation-datasets/)

Thus the original dataset is generally split in a training, validation and a test data sets. Large training+validation set (80%) small test set (20%) might provide a poor estimation of the predictive performances (same argument stands for train vs validation samples). On the contrary, large test set and small training set might produce a poorly estimated learner. This is why, on situation where we cannot afford such split, it recommended to use cross-validation scheme to estimate the predictive power of a learning algorithm.


## Cross-Validation (CV)

Cross-Validation scheme randomly divides the set of observations into $K$ groups, or **folds**, of approximately equal size. The first fold is treated as a validation set, and the method $f()$ is fitted on the remaining union of $K - 1$ folds: ($f(\boldsymbol{X}_{-K}, \boldsymbol{y}_{-K})$).

The measure of performance (the score function $\mathcal{S}$), either a error measure or an correct prediction measure is an average of a loss error or correct prediction measure, noted $\mathcal{L}$, between a true target value and the predicted target value. The score function is evaluated of the on the observations in the held-out fold. For each sample $i$ we consider the model estimated $f(\boldsymbol{X}_{-k(i)}, \boldsymbol{y}_{-k(i)}$ on the data set without the group $k$ that contains $i$ noted $-k(i)$. This procedure is repeated $K$ times; each time, a different group of observations is treated as a test set.
Then we compare the predicted value ($f_{-k(i)}(\boldsymbol{x}_i) = \hat{y_i})$ with true value $y_i$ using a Error or Loss function $\mathcal{L}(y, \hat{y})$.

For 10-fold we can either average over 10 values (Macro measure) or concatenate the 10 experiments and compute the micro measures.

Two strategies [micro vs macro estimates](https://stats.stackexchange.com/questions/34611/meanscores-vs-scoreconcatenation-in-cross-validation):


**Micro measure: average(individual scores)**: compute a score $\mathcal{S}$ for each sample and average over all samples. It is simillar to **average score(concatenation)**: an averaged score computed over all concatenated samples.
$$
\mathcal{S}(f) = \frac{1}{N} \sum_i^N \mathcal{L}\left(y_i, f(\boldsymbol{x}_{-k(i)}, \boldsymbol{y}_{-k(i)}) \right).
$$


**Macro measure mean(CV scores)** (the most commonly used method): compute a score $\mathcal{S}$ on each each fold $k$ and average accross folds:
 
\begin{align*}
\mathcal{S}(f) &= \frac{1}{K} \sum_k^K \mathcal{S}_k(f).\\
\mathcal{S}(f) &= \frac{1}{K} \sum_k^K \frac{1}{N_k} \sum_{i \in k} \mathcal{L}\left(y_i, f(\boldsymbol{x}_{-k(i)}, \boldsymbol{y}_{-k(i)}) \right).
\end{align*}

These two measures (an average of average vs. a global average) are generaly similar. They may differ slightly is folds are of different sizes.

This validation scheme is known as the **K-Fold CV**. Typical choices of $K$ are 5 or 10, [Kohavi 1995]. The extreme case where $K = N$ is known as **leave-one-out cross-validation, LOO-CV**.

### CV for regression

Usually the error function $\mathcal{L}()$ is the r-squared score. However other function could be used.

In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings(action='once')

In [ ]:
import numpy as np
from sklearn import datasets
import sklearn.linear_model as lm
import sklearn.metrics as metrics
from sklearn.model_selection import KFold

X, y = datasets.make_regression(n_samples=100, n_features=100, 
                         n_informative=10, random_state=42)
estimator = lm.Ridge(alpha=10)

cv = KFold(n_splits=5, random_state=42)
r2_train, r2_test = list(), list()

for train, test in cv.split(X):
    estimator.fit(X[train, :], y[train])
    r2_train.append(metrics.r2_score(y[train], estimator.predict(X[train, :])))
    r2_test.append(metrics.r2_score(y[test], estimator.predict(X[test, :])))    

print("Train r2:%.2f" % np.mean(r2_train))
print("Test  r2:%.2f" % np.mean(r2_test))

Scikit-learn provides user-friendly function to perform CV:

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=estimator, X=X, y=y, cv=5)
print("Test  r2:%.2f" % scores.mean())

# provide a cv
cv = KFold(n_splits=5, random_state=42)
scores = cross_val_score(estimator=estimator, X=X, y=y, cv=cv)
print("Test  r2:%.2f" % scores.mean())

### CV for classification

With classification problems it is essential to sample folds where each set contains approximately the same percentage of samples of each target class as the complete set. This is called **stratification**. In this case, we will use ``StratifiedKFold`` with is a variation of k-fold which returns stratified folds.

Usually the error function $L()$ are, at least, the sensitivity and the specificity. However other function could be used.

In [ ]:
import numpy as np
from sklearn import datasets
import sklearn.linear_model as lm
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedKFold

X, y = datasets.make_classification(n_samples=100, n_features=100, 
                         n_informative=10, random_state=42)

estimator = lm.LogisticRegression(C=1, solver='lbfgs')

cv = StratifiedKFold(n_splits=5)

# Lists to store scores by folds (for macro measure only)
recalls_train, recalls_test, acc_test = list(), list(), list()

# Or vector of test predictions (for both macro and micro measures, not for training samples)
y_test_pred = np.zeros(len(y))

for train, test in cv.split(X, y):
    estimator.fit(X[train, :], y[train])
    recalls_train.append(metrics.recall_score(y[train], estimator.predict(X[train, :]), average=None))
    recalls_test.append(metrics.recall_score(y[test], estimator.predict(X[test, :]), average=None))
    acc_test.append(metrics.accuracy_score(y[test], estimator.predict(X[test, :])))

    # Store test predictions (for micro measures)
    y_test_pred[test] = estimator.predict(X[test, :])

print("== Macro measures ==")
# Use lists of scores
recalls_train = np.array(recalls_train)
recalls_test = np.array(recalls_test)
print("Train SPC:%.2f; SEN:%.2f" % tuple(recalls_train.mean(axis=0)))
print("Test  SPC:%.2f; SEN:%.2f" % tuple(recalls_test.mean(axis=0)), )
print("Test  ACC:%.2f, ballanced ACC:%.2f" %
      (np.mean(acc_test), recalls_test.mean(axis=1).mean()), "Folds:", acc_test)

# Or use vector to test predictions
acc_test = [metrics.accuracy_score(y[test], y_test_pred[test]) for train, test in cv.split(X, y)]
print("Test  ACC:%.2f" % np.mean(acc_test), "Folds:", acc_test)

print("== Micro measures ==")
print("Test SPC:%.2f; SEN:%.2f" % \
      tuple(metrics.recall_score(y, y_test_pred, average=None)))
print("Test  ACC:%.2f" % metrics.accuracy_score(y, y_test_pred))

Scikit-learn provides user-friendly function to perform CV:

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(estimator=estimator, X=X, y=y, cv=5)
scores.mean()

# provide CV and score
def balanced_acc(estimator, X, y, **kwargs):
    '''
    Balanced acuracy scorer
    '''
    return metrics.recall_score(y, estimator.predict(X), average=None).mean()

scores = cross_val_score(estimator=estimator, X=X, y=y, cv=5, scoring=balanced_acc)
print("Test  ACC:%.2f" % scores.mean())

Note that with Scikit-learn user-friendly function we average the scores' average obtained on individual folds which may provide slightly different results that the overall average presented earlier.

## Parallel computation with joblib

Dataset

In [ ]:
import numpy as np
from sklearn import datasets
import sklearn.linear_model as lm
import sklearn.metrics as metrics
from sklearn.model_selection import StratifiedKFold
X, y = datasets.make_classification(n_samples=20, n_features=5, n_informative=2, random_state=42)
cv = StratifiedKFold(n_splits=5)

### Use `cross_validate` function

In [ ]:
from sklearn.model_selection import cross_validate

estimator = lm.LogisticRegression(C=1, solver='lbfgs')
cv_results = cross_validate(estimator, X, y, cv=cv, n_jobs=5)
print(np.mean(cv_results['test_score']), cv_results['test_score'])

### Sequential computation

If we want have full control of the operations performed within each fold (retrieve the models parameters, etc.). We would like to parallelize the folowing sequetial code:

In [ ]:
estimator = lm.LogisticRegression(C=1, solver='lbfgs')
y_test_pred_seq = np.zeros(len(y)) # Store predictions in the original order
coefs_seq = list()
for train, test in cv.split(X, y):
    X_train, X_test, y_train, y_test = X[train, :], X[test, :], y[train], y[test]
    estimator.fit(X_train, y_train)
    y_test_pred_seq[test] = estimator.predict(X_test)
    coefs_seq.append(estimator.coef_)

test_accs = [metrics.accuracy_score(y[test], y_test_pred_seq[test]) for train, test in cv.split(X, y)]
print(np.mean(test_accs), test_accs)
coefs_cv = np.array(coefs_seq)
print(coefs_cv)

print(coefs_cv.mean(axis=0))
print("Std Err of the coef")
print(coefs_cv.std(axis=0) / np.sqrt(coefs_cv.shape[0]))

### Parallel computation with joblib

In [ ]:
from sklearn.externals.joblib import Parallel, delayed
from sklearn.base import is_classifier, clone

def _split_fit_predict(estimator, X, y, train, test):
    X_train, X_test, y_train, y_test = X[train, :], X[test, :], y[train], y[test]
    estimator.fit(X_train, y_train)
    return [estimator.predict(X_test), estimator.coef_]

estimator = lm.LogisticRegression(C=1, solver='lbfgs')

parallel = Parallel(n_jobs=5)
cv_ret = parallel(
    delayed(_split_fit_predict)(
        clone(estimator), X, y, train, test)
    for train, test in cv.split(X, y))

y_test_pred_cv, coefs_cv = zip(*cv_ret)

# Retrieve predictions in the original order
y_test_pred = np.zeros(len(y))
for i, (train, test) in enumerate(cv.split(X, y)):
    y_test_pred[test] = y_test_pred_cv[i]

test_accs = [metrics.accuracy_score(y[test], y_test_pred[test]) for train, test in cv.split(X, y)]
print(np.mean(test_accs), test_accs)

Test same predictions and same coeficients

In [ ]:
assert np.all(y_test_pred == y_test_pred_seq)
assert np.allclose(np.array(coefs_cv).squeeze(), np.array(coefs_seq).squeeze())

## CV for model selection: setting the hyper parameters

It is important to note CV may be used for two separate goals:

1. **Model assessment**: having chosen a final model, estimating its prediction error (generalization error) on new data.

2. **Model selection**: estimating the performance of different models in order to choose the best one. One special case of model selection is the selection model's hyper parameters. Indeed remember that most of learning algorithm have a hyper parameters (typically the regularization parameter) that has to be set.

Generally we must address the two problems simultaneously. The usual approach for both problems is to randomly divide the dataset into three parts: a training set, a validation set, and a test set.

- The **training set** (train) is used to fit the models;

- the **validation set** (val) is used to estimate prediction error for model selection or to determine the hyper parameters over a grid of possible values.

- the **test set** (test) is used for assessment of the generalization error of the final chosen model.


### Grid search procedure

Model selection of the best hyper parameters over a grid of possible values

For each possible values of hyper parameters $\alpha_k$:

1. Fit the learner on training set: $f(X_{train}, y_{train}, \alpha_k)$

2. Evaluate the model on the validation set and keep the parameter(s) that minimises the error measure

    $\alpha_* = \arg \min L(f(X_{train}), y_{val}, \alpha_k)$

3. Refit the learner on all training + validation data using the best hyper parameters: $f^* \equiv f(X_{train \cup val}, y_{train \cup val}, \alpha_*)$

4. ** Model assessment ** of $f^*$ on the test set: $L(f^*(X_{test}), y_{test})$

### Nested CV for model selection and assessment

Most of time, we cannot afford such three-way split. Thus, again we will use CV, but in this case we need two nested CVs.

One **outer CV loop, for model assessment**. This CV performs $K$ splits of the dataset into training plus validation ($X_{-K}, y_{-K}$) set and a test set $X_{K}, y_{K}$

One **inner CV loop, for model selection**. For each run of the outer loop, the inner loop loop performs $L$ splits of dataset ($X_{-K}, y_{-K}$) into training set: ($X_{-K,-L}, y_{-K,-L}$) and a validation set: ($X_{-K,L}, y_{-K,L}$).

### Implementation with scikit-learn

Note that the inner CV loop combined with the learner form a new learner with an automatic model (parameter) selection procedure. This new learner can be easily constructed using Scikit-learn. The learned is wrapped inside a ``GridSearchCV`` class.

Then the new learned can be plugged into the classical outer CV loop.

In [ ]:
import numpy as np
from sklearn import datasets
import sklearn.linear_model as lm
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
from sklearn.model_selection import KFold

# Dataset
noise_sd = 10
X, y, coef = datasets.make_regression(n_samples=50, n_features=100, noise=noise_sd,
                         n_informative=2, random_state=42, coef=True)
 
# Use this to tune the noise parameter such that snr < 5
print("SNR:", np.std(np.dot(X, coef)) / noise_sd)

# param grid over alpha & l1_ratio
param_grid = {'alpha': 10. ** np.arange(-3, 3), 'l1_ratio':[.1, .5, .9]}

# Warp 
model = GridSearchCV(lm.ElasticNet(max_iter=10000), param_grid, cv=5)

### Regression models with built-in cross-validation

Sklearn will automatically select a grid of parameters, most of time use the defaults values.

``n_jobs`` is the number of CPUs to use during the cross validation. If -1, use all the CPUs.


1) Biased usage: fit on all data, ommit outer CV loop                 

In [ ]:
model.fit(X, y)
print("Train r2:%.2f" % metrics.r2_score(y, model.predict(X)))
print(model.best_params_)

2) User made outer CV, useful to extract specific information 

In [ ]:
cv = KFold(n_splits=5, random_state=42)
r2_train, r2_test = list(), list() 
alphas = list()

for train, test in cv.split(X, y):
    X_train, X_test, y_train, y_test = X[train, :], X[test, :], y[train], y[test]
    model.fit(X_train, y_train)
    
    r2_test.append(metrics.r2_score(y_test, model.predict(X_test)))
    r2_train.append(metrics.r2_score(y_train, model.predict(X_train)))

    alphas.append(model.best_params_)

print("Train r2:%.2f" % np.mean(r2_train))
print("Test  r2:%.2f" % np.mean(r2_test))
print("Selected alphas:", alphas)

3) User-friendly sklearn for outer CV

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=model, X=X, y=y, cv=cv)
print("Test  r2:%.2f" % scores.mean())

In [ ]:
from sklearn import datasets
import sklearn.linear_model as lm
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score

# Dataset
X, y, coef = datasets.make_regression(n_samples=50, n_features=100, noise=10,
                         n_informative=2, random_state=42, coef=True)
 

print("== Ridge (L2 penalty) ==")
model = lm.RidgeCV(cv=3)
# Let sklearn select a list of alphas with default LOO-CV
scores = cross_val_score(estimator=model, X=X, y=y, cv=5)
print("Test  r2:%.2f" % scores.mean())

print("== Lasso (L1 penalty) ==")
model = lm.LassoCV(n_jobs=-1, cv=3)
# Let sklearn select a list of alphas with default 3CV
scores = cross_val_score(estimator=model, X=X, y=y, cv=5)
print("Test  r2:%.2f" % scores.mean())

print("== ElasticNet (L1 penalty) ==")
model = lm.ElasticNetCV(l1_ratio=[.1, .5, .9], n_jobs=-1, cv=3)
# Let sklearn select a list of alphas with default 3CV
scores = cross_val_score(estimator=model, X=X, y=y, cv=5)
print("Test  r2:%.2f" % scores.mean())

### Classification models with built-in cross-validation

In [ ]:
from sklearn import datasets
import sklearn.linear_model as lm
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score

X, y = datasets.make_classification(n_samples=100, n_features=100, 
                         n_informative=10, random_state=42)

# provide CV and score
def balanced_acc(estimator, X, y, **kwargs):
    '''
    Balanced accuracy scorer
    '''
    return metrics.recall_score(y, estimator.predict(X), average=None).mean()

print("== Logistic Ridge (L2 penalty) ==")
model = lm.LogisticRegressionCV(class_weight='balanced', scoring=balanced_acc, n_jobs=-1, cv=3)
# Let sklearn select a list of alphas with default LOO-CV
scores = cross_val_score(estimator=model, X=X, y=y, cv=5)
print("Test  ACC:%.2f" % scores.mean())

## Random Permutations

A permutation test is a type of non-parametric randomization test in which the null distribution of a test statistic is estimated by randomly permuting the observations.

Permutation tests are highly attractive because they make no assumptions other than that the observations are independent and identically distributed under the null hypothesis.

1. Compute a observed statistic $t_{obs}$ on the data. 
2. Use randomization to compute the distribution of $t$ under the null hypothesis: Perform $N$ random permutation of the data. For each sample of permuted data, $i$ the data compute the statistic $t_i$. This procedure provides the distribution of $t$ under the null hypothesis $H_0$: $P(t \vert H_0)$
3. Compute the p-value = $P(t>t_{obs} | H_0) \left\vert\{t_i > t_{obs}\}\right\vert$, where $t_i$'s include $t_{obs}$.

### Example with a correlation

The statistic is the correlation.

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#%matplotlib qt

np.random.seed(42)
x = np.random.normal(loc=10, scale=1, size=100)
y = x + np.random.normal(loc=-3, scale=3, size=100) # snr = 1/2

# Permutation: simulate the null hypothesis
nperm = 10000
perms = np.zeros(nperm + 1)

perms[0] = np.corrcoef(x, y)[0, 1]

for i in range(1, nperm):
    perms[i] = np.corrcoef(np.random.permutation(x), y)[0, 1]

# Plot
# Re-weight to obtain distribution
weights = np.ones(perms.shape[0]) / perms.shape[0]
plt.hist([perms[perms >= perms[0]], perms], histtype='stepfilled', 
         bins=100, label=["t>t obs (p-value)", "t<t obs"], 
         weights=[weights[perms >= perms[0]], weights])

plt.xlabel("Statistic distribution under null hypothesis")
plt.axvline(x=perms[0], color='blue', linewidth=1, label="observed statistic")
_ = plt.legend(loc="upper left")

# One-tailed empirical p-value
pval_perm = np.sum(perms >= perms[0]) / perms.shape[0]

# Compare with Pearson's correlation test
_, pval_test = stats.pearsonr(x, y)

print("Permutation two tailed p-value=%.5f. Pearson test p-value=%.5f" % (2*pval_perm, pval_test))

### Exercise

Given the logistic regression presented above and its validation given a 5 folds CV.

1. Compute the p-value associated with the prediction accuracy using a permutation test.

2. Compute the p-value associated with the prediction accuracy using a parametric test.

## Bootstrapping


Bootstrapping is a random sampling with replacement strategy which provides an non-parametric method to assess the variability of performances scores such standard errors or [confidence intervals.](https://sebastianraschka.com/blog/2016/model-evaluation-selection-part2.html#the-bootstrap-method-and-empirical-confidence-intervals)

A great advantage of bootstrap is its simplicity. It is a straightforward way to derive estimates of standard errors and confidence intervals for complex estimators of complex parameters of the distribution, such as percentile points, proportions, odds ratio, and correlation coefficients. 

1. Perform $B$ sampling, with replacement, of the dataset.
2. For each sample $i$ fit the model and compute the scores.
3. Assess standard errors and confidence intervals of scores using the scores obtained on the $B$ resampled dataset.

In [ ]:
import numpy as np
from sklearn import datasets
import sklearn.linear_model as lm
import sklearn.metrics as metrics
import pandas as pd

# Regression dataset
n_features = 5
n_features_info = 2
n_samples = 100
X = np.random.randn(n_samples, n_features)
beta = np.zeros(n_features)
beta[:n_features_info] = 1
Xbeta = np.dot(X, beta)
eps = np.random.randn(n_samples)
y = Xbeta + eps

# Fit model on all data (!! risk of overfit)
model = lm.RidgeCV()
model.fit(X, y)
print("Coefficients on all data:")
print(model.coef_)

# Bootstrap loop
nboot = 100  # !! Should be at least 1000
scores_names = ["r2"]
scores_boot = np.zeros((nboot, len(scores_names)))
coefs_boot = np.zeros((nboot, X.shape[1]))

orig_all = np.arange(X.shape[0])
for boot_i in range(nboot):
    boot_tr = np.random.choice(orig_all, size=len(orig_all), replace=True)
    boot_te = np.setdiff1d(orig_all, boot_tr, assume_unique=False)
    Xtr, ytr = X[boot_tr, :], y[boot_tr]
    Xte, yte = X[boot_te, :], y[boot_te]
    model.fit(Xtr, ytr)
    y_pred = model.predict(Xte).ravel()
    scores_boot[boot_i, :] = metrics.r2_score(yte, y_pred)
    coefs_boot[boot_i, :] = model.coef_

# Compute Mean, SE, CI
scores_boot = pd.DataFrame(scores_boot, columns=scores_names)
scores_stat = scores_boot.describe(percentiles=[.975, .5, .025])

print("r-squared: Mean=%.2f, SE=%.2f, CI=(%.2f %.2f)" %\
      tuple(scores_stat.loc[["mean", "std", "5%", "95%"], "r2"]))

coefs_boot = pd.DataFrame(coefs_boot)
coefs_stat = coefs_boot.describe(percentiles=[.975, .5, .025])
print("Coefficients distribution")
print(coefs_stat)